In [96]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
import time
import concurrent.futures
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os


In [97]:
file_path = 'Data\Binance_BTCUSDT_2024_minute.csv'
df = pd.read_csv(file_path)
#print(df.head())

In [98]:
df = df.drop('unix', axis=1).drop('date', axis=1).drop('symbol', axis=1).drop('volume_from', axis=1) 
# df = df.drop('marketorder_volume_from', axis=1).drop('marketorder_volume', axis=1).drop('date_close', axis=1).drop('close_unix', axis=1) 
#df

In [99]:
df = df.iloc[::-1]
df

,open,high,low,close,volume,tradecount
69071,42298.62,42320.00,42298.61,42320.00,21.16779,1348
69070,42319.99,42331.54,42319.99,42325.50,21.60391,1019
69069,42325.50,42368.00,42325.49,42367.99,30.50730,1241
69068,42368.00,42397.23,42367.99,42397.23,46.05107,1415
69067,42397.22,42409.20,42385.26,42409.20,32.26766,1255
...,...,...,...,...,...,...
4,51849.99,51888.89,51849.99,51870.64,36.86096,653
3,51870.63,51870.64,51841.10,51841.10,11.01314,495
2,51841.11,51847.32,51841.10,51847.31,11.83013,423
1,51847.31,51851.19,51829.20,51829.20,18.49469,506


In [100]:
df_natural = df

In [101]:
# df = df.dropna()

# # Iterate over each column except for non-numeric columns
# for column in df.columns:
#     if pd.api.types.is_numeric_dtype(df[column]):
#         df[column] = df[column].pct_change() * 100

# df = df.dropna()

In [102]:
def create_neural_network(input_dim: int) -> Sequential:
    model = Sequential()
    model.add(Dense(16, input_dim=input_dim, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(3, activation='softmax')) # Output layer: 3 nodes for buy, sell, wait
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [103]:
def create_lstm_model(timesteps, features):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(timesteps, features)))
    model.add(LSTM(50))
    model.add(Dense(25))
    model.add(Dense(3, activation='softmax'))  # For 3 output classes
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [104]:
def handle_buy_action(trading_state, current_price, current_index):
    trading_state['openPrice'] = current_price
    trading_state['inPosition'] = True
    trading_state['entryIndex'] = current_index

def handle_sell_action(trading_state, current_price, current_index, loss, hold_time):
    profit = (current_price - trading_state['openPrice'])
    commission = abs(profit) * 0.002
    profit -= commission

    trade_details = {
        'Open': trading_state['openPrice'],
        'Close': current_price,
        'Profit': profit,
        'Candles In Position': hold_time,
        'Open Index': trading_state['entryIndex'],
        'Close Index': current_index,
        'Closed By': 'Stop Loss' if loss >= 0.1 else 'Model'
    }

    trading_state['inPosition'] = False
    return profit, trade_details


In [105]:
class TradingState:
    def __init__(self):
        self.inPosition = False
        self.openPrice = 0
        self.entryTime = 0
        self.closeReason = None  # 'model' or 'stop_loss'

    def enter_position(self, open_price, current_time):
        self.inPosition = True
        self.openPrice = open_price
        self.entryTime = current_time
        self.closeReason = None

    def exit_position(self, reason):
        self.inPosition = False
        self.closeReason = reason

    def time_in_trade(self, current_time):
        return current_time - self.entryTime



def make_trade_decision(current_state: TradingState, current_price, prediction, stop_loss_threshold, current_time):
    action = np.argmax(prediction)
    profit = 0

    # Check if currently in a position
    if current_state.inPosition:
        # Calculate the loss percentage
        loss_percent = (current_price - current_state.openPrice) / current_state.openPrice

        # Check for stop loss condition
        if loss_percent <= -stop_loss_threshold:
            action = 1  # Override action to sell due to stop loss
            current_state.closeReason = 'stop_loss'

        # Check for sell action
        if action == 1:
            profit = calculate_profit(current_state, current_price)
            current_state.closeReason = current_state.closeReason or 'model'
            current_state.exit_position(current_state.closeReason)

    # Check for buy action
    elif action == 0:
        current_state.enter_position(current_price, current_time)

    return profit

def calculate_profit(current_state: TradingState, current_price):
    profit = current_price - current_state.openPrice
    commission = abs(profit) * 0.002
    return profit - commission


def trade_with_model(models: list[Sequential], data: pd.DataFrame, stop_loss_threshold: float = 0.2) -> list[float]:
    data_np = data.to_numpy()
    all_predictions = [model.predict(data_np, batch_size=128) for model in models]

    trading_states = [TradingState() for _ in models]
    profits_by_model = [0 for _ in models]
    trades_by_model = [0 for _ in models]

    for i in range(len(data_np)):
        current_price = data_np[i][data.columns.get_loc('close')]
        for model_index, predictions in enumerate(all_predictions):
            profit = make_trade_decision(
                trading_states[model_index], 
                current_price, 
                predictions[i], 
                stop_loss_threshold, 
                i
            )
            profits_by_model[model_index] += profit
            if profit != 0:
                trades_by_model[model_index] += 1

    return profits_by_model, trades_by_model


def test_single_model(model: Sequential, data: pd.DataFrame, stop_loss_threshold: float = 0.2) -> pd.DataFrame:
    data_np = data.to_numpy()
    predictions = model.predict(data_np, batch_size=128)

    trading_state = TradingState()
    trades_info = []

    for i in range(len(data_np)):
        current_price = data_np[i][data.columns.get_loc('close')]
        profit = make_trade_decision(
            trading_state, 
            current_price, 
            predictions[i], 
            stop_loss_threshold, 
            i
        )
        if trading_state.inPosition == False and trading_state.closeReason is not None:
            trade = {
                'open_index': trading_state.entryTime,
                'open_price': trading_state.openPrice,
                'close_index': i,
                'close_price': current_price,
                'profit': profit,
                'candles_in_position': i - trading_state.entryTime,
                'closed_by': trading_state.closeReason
            }
            trades_info.append(trade)
            trading_state.closeReason = None  # Reset reason after recording the trade

    trades_df = pd.DataFrame(trades_info)
    return trades_df



In [106]:
def crossover_and_mutate(model1: Sequential, model2: Sequential, input_dim: int, mutation_rate=0.1, mutation_scale=0.1) -> Sequential:
    print("Performing crossover and mutation...")
    
    # Retrieve weights from both models
    weights1 = model1.get_weights()
    weights2 = model2.get_weights()

    # New weights list
    new_weights = []

    # Randomly mix weights from both parents and mutate
    for w1, w2 in zip(weights1, weights2):
        mask = np.random.randint(0, 2, size=w1.shape)
        new_w = np.where(mask, w1, w2)

        # Mutation: Apply random changes to weights
        if np.random.rand() < mutation_rate:
            mutation = np.random.normal(loc=0.0, scale=mutation_scale, size=new_w.shape)
            new_w += mutation

        new_weights.append(new_w)

    # Create a new model and set the mixed and mutated weights
    new_model = create_neural_network(input_dim)
    new_model.set_weights(new_weights)

    return new_model


In [107]:
def evaluate_model(models: list[Sequential], data: pd.DataFrame) -> list[float]:
    performances, trades = trade_with_model(models, data)
    for i in range(len(performances)):
        print(f"Model evaluated with performance: {performances[i]}, trades: {trades[i]}")    
    return performances

In [108]:
def genetic_algorithm(initial_population: list[Sequential], data: pd.DataFrame, generations: int) -> list[Sequential]:
    population = initial_population
    for generation in range(generations):
        print(f"Generation {generation + 1}/{generations}")

        # Evaluate each model
        performances = evaluate_model(population, data)
        
        # Sort and select top models
        sorted_population = [model for _, model in sorted(zip(performances, population), key=lambda pair: pair[0], reverse=True)]
        num_top_models = len(population) // 2
        top_models = sorted_population[:num_top_models]
        print(f"Selected top {num_top_models} models for next generation.")

        # Breed and mutate new models from top models
        new_models = []
        for i in range(len(population) - num_top_models):
            parents = np.random.choice(top_models, 2)
            child = crossover_and_mutate(parents[0], parents[1], data.shape[1], mutation_rate=0.5, mutation_scale=0.5)
            new_models.append(child)

        # Next generation
        population = top_models + new_models
        print(f"Generation {generation + 1} complete.")

    return population


In [109]:
def save_models(models: list[Sequential], folder: str = "Models"):
    if not os.path.exists(folder):
        os.makedirs(folder)

    for i, model in enumerate(models):
        model_path = os.path.join(folder, f"model_{i + 1}.h5")
        model.save(model_path)
        print(f"Model {i + 1} saved at {model_path}")

In [110]:
from keras.models import load_model
def load_models(model_names: list[str], folder: str = "Models") -> list[Sequential]:
    loaded_models = []
    
    for model_name in model_names:
        model_path = os.path.join(folder, model_name)
        if os.path.exists(model_path):
            model = load_model(model_path)
            loaded_models.append(model)
            print(f"Loaded model from {model_path}")
        else:
            print(f"Model file {model_path} not found.")

    return loaded_models


In [111]:
population = [create_neural_network(df.shape[1]) for _ in range(5)] + load_models(["model_1.h5", "model_2.h5", "model_3.h5"])
final_population = genetic_algorithm(population, df, generations=20)

evaluate_model(final_population, df)

Loaded model from Models\model_1.h5
Loaded model from Models\model_2.h5
Loaded model from Models\model_3.h5
Generation 1/20
540/540 [==============================] - 1s 899us/step
Model evaluated with performance: 0, trades: 0
Model evaluated with performance: 0, trades: 0
Model evaluated with performance: 0, trades: 0
Model evaluated with performance: 9522.149999999996, trades: 61
Model evaluated with performance: 0, trades: 0
Model evaluated with performance: 2231.422159999958, trades: 354
Model evaluated with performance: 1420.4710400000042, trades: 102
Model evaluated with performance: 912.0301400000009, trades: 7
Selected top 4 models for next generation.
Performing crossover and mutation...
Performing crossover and mutation...
Performing crossover and mutation...
Performing crossover and mutation...
Generation 1 complete.
Generation 2/20
540/540 [==============================] - 1s 927us/step
Model evaluated with performance: 9522.149999999996, trades: 61
Model evaluated with p

[10589.920259999986,
 10589.920259999986,
 10589.920259999986,
 9522.149999999996,
 -6491.721860000004,
 3328.379900000003,
 0,
 0]

In [112]:
# population = []

# while len(population) < 3:
#     model = create_neural_network(df.shape[1])
#     model_perf = evaluate_model([model], df)[0]
#     if model_perf > 0:
#         population.append(model)



In [113]:

def trade_with_single_model(model: Sequential, data: pd.DataFrame):
    data_np = data.to_numpy()
    predictions = model.predict(data_np, batch_size=128)

    trading_state = {'inPosition': False, 'openPrice': 0, 'entryIndex': -1}
    total_profit = 0

    trade_records = []  # List to store records of each trade

    for i in range(len(data_np)):
        current_price = data_np[i][data.columns.get_loc('close')]
        prediction = predictions[i]
        action = np.argmax(prediction)
        hold_time = i - trading_state['entryIndex']

        # Check if we're in position and need to act
        if trading_state['inPosition']:
            loss = (trading_state['openPrice'] - current_price) / trading_state['openPrice']
            if action == 1 or loss >= 0.1:  # Action to sell or stop loss triggered
                profit, trade_details = handle_sell_action(trading_state, current_price, i, loss, hold_time)
                total_profit += profit
                trade_records.append(trade_details)

        # Buy action
        if action == 0 and not trading_state['inPosition']:
            handle_buy_action(trading_state, current_price, i)

    # Create a DataFrame with the trading results
    results_df = pd.DataFrame(trade_records)
    print(results_df)
    # Plotting
    buffer = max(data['close']) * 0.005
    plt.figure(figsize=(12, 6))
    plt.plot(data['close'], label='Close Price', color='blue')
    plt.scatter(results_df['Open Index'], data['close'][results_df['Open Index']], label='Buy', marker='^', color='green', zorder=5)
    plt.scatter(results_df['Close Index'], data['close'][results_df['Close Index']], label='Sell', marker='v', color='red', zorder=5)
    plt.title('Trading Strategy Visualization')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    return results_df

In [124]:
#partial_df = df[10000:17000].reset_index(drop=True)
result = test_single_model(final_population[3], df)#.sort_values('close_', ascending=False)
result

540/540 [==============================] - 1s 971us/step


,open_index,open_price,close_index,close_price,profit,candles_in_position,closed_by
0,0,42320.00,1495,45036.00,2710.56800,1495,model
1,1496,45022.76,3561,44535.48,-488.25456,2065,model
2,3562,44567.02,3592,43614.72,-954.20460,30,model
3,3593,43663.04,3598,42573.63,-1091.58882,5,model
4,3599,42908.00,3605,41959.68,-950.21664,6,model
...,...,...,...,...,...,...,...
56,55615,49391.73,55720,50041.09,648.06128,105,model
57,55721,49862.01,56982,48773.59,-1090.59684,1261,model
58,56983,48539.60,58097,50648.02,2104.20316,1114,model
59,58098,50758.00,59869,52600.01,1838.32598,1771,model


In [125]:
result.describe()

,open_index,open_price,close_index,close_price,profit,candles_in_position
count,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000
mean,21518.114754,44315.640492,22615.311475,44473.435246,156.100820,1097.196721
std,17677.440081,2821.797339,18385.954223,3007.120091,1104.614746,2202.398176
min,0.000000,39270.430000,1495.000000,39402.010000,-2665.299960,1.000000
25%,10454.000000,42375.700000,10787.000000,42431.100000,-692.271780,29.000000
50%,14163.000000,44230.000000,14165.000000,44490.000000,151.127140,230.000000
75%,30853.000000,45776.980000,30858.000000,45816.720000,699.598000,1204.000000
max,59870.000000,52665.470000,67007.000000,52612.750000,3710.623880,13483.000000


In [ ]:
save_models(population)

Model 1 saved at Models\model_1.h5
Model 2 saved at Models\model_2.h5
Model 3 saved at Models\model_3.h5


In [ ]:
saved_models = load_models(["model_1.h5", "model_2.h5", "model_3.h5"])

Loaded model from Models\model_1.h5
Loaded model from Models\model_2.h5
Loaded model from Models\model_3.h5


In [ ]:
evaluate_model(saved_models, df)

4099/4099 [==============================] - 4s 901us/step
Model evaluated with performance: 33885.02051999993, trades: 2138
Model evaluated with performance: 1656.38583999995, trades: 333
Model evaluated with performance: 17999.014679999993, trades: 17


[33885.02051999993, 1656.38583999995, 17999.014679999993]